# From
https://gist.github.com/TomNicholas/ee5c673f8772beeecd3f58f9b040943f#file-hero_calc_delayed-ipynb

### Install packages

## Define Regions

In [1]:
import xarray as xr
import numpy as np

ds_grid = xr.open_dataset(
    "gs://pangeo-ecco-llc4320/grid", 
    engine="zarr", 
    storage_options={'requester_pays': True},
)
lat = ds_grid['YC'].reset_coords(drop=True).rename('lat')
lon = ds_grid['XC'].reset_coords(drop=True).rename('lon')
depth = ds_grid['Depth'].reset_coords(drop=True)
depth

<xarray.DataArray 'Depth' (face: 13, j: 4320, i: 4320)>
[242611200 values with dtype=float32]
Coordinates:
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
Attributes:
    coordinate:     XC YC
    long_name:      ocean depth
    standard_name:  ocean_depth
    units:          m

In [2]:
region_shape = (540, 540)

i_max = (depth.i.coarsen(i=region_shape[1]).max() + 1).rename('i_max').drop('i').astype('i2')
i_min = depth.i.coarsen(i=region_shape[1]).min().rename('i_min').drop('i').astype('i2')
j_max = (depth.j.coarsen(j=region_shape[0]).max() + 1).rename('j_max').drop('j').astype('i2')
j_min = (depth.j.coarsen(j=region_shape[0]).min()).rename('j_min').drop('j').astype('i2')

lat_max = lat.coarsen(j=region_shape[0], i=region_shape[1]).max().rename('lat_max').drop(['i', 'j'])
lat_min = lat.coarsen(j=region_shape[0], i=region_shape[1]).min().rename('lat_min').drop(['i', 'j'])
lon_max = lon.coarsen(j=region_shape[0], i=region_shape[1]).max().rename('lon_max').drop(['i', 'j'])
lon_min = lon.coarsen(j=region_shape[0], i=region_shape[1]).min().rename('lon_min').drop(['i', 'j'])

rotation_rate = 7.2921e-5  # rad/s
f_coriolis = (
    2 * rotation_rate * np.sin(lat*np.pi/180)
).coarsen(j=region_shape[0], i=region_shape[1]).mean().rename('f_coriolis').drop(['i', 'j'])
f_coriolis

mask = (depth > 100).rename('mask').drop(['i', 'j'])
fraction = mask.coarsen(j=region_shape[0], i=region_shape[1]).mean().rename('area_fraction')

regions = xr.merge([lat_max, lat_min, lon_max, lon_min, fraction, f_coriolis, i_min, i_max, j_min, j_max])
regions

<xarray.Dataset>
Dimensions:        (face: 13, j: 8, i: 8)
Coordinates:
  * face           (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
Dimensions without coordinates: j, i
Data variables:
    lat_max        (face, j, i) float32 -86.8 -87.05 -86.08 ... -83.3 -85.71
    lat_min        (face, j, i) float32 -89.75 -90.0 -88.67 ... -86.79 -88.17
    lon_max        (face, j, i) float32 -30.56 64.91 64.96 ... -45.51 -61.2
    lon_min        (face, j, i) float32 -114.9 -114.5 2.951 ... -79.91 -115.0
    area_fraction  (face, j, i) float64 0.0 0.0 0.0 0.0 ... 0.3923 0.0 0.0 0.0
    f_coriolis     (face, j, i) float32 -0.0001458 -0.0001458 ... -0.0001456
    i_min          (i) int16 0 540 1080 1620 2160 2700 3240 3780
    i_max          (i) int16 540 1080 1620 2160 2700 3240 3780 4320
    j_min          (j) int16 0 540 1080 1620 2160 2700 3240 3780
    j_max          (j) int16 540 1080 1620 2160 2700 3240 3780 4320
Attributes:
    coordinate:     YC XC
    long_name:      latitude
    standard_name:  latitude
    units:          degrees_north

In [3]:
del depth, lat, lon # try to free some memory

In [4]:
region_df = regions.to_dataframe().reset_index()
region_df

,face,j,i,lat_max,lat_min,lon_max,lon_min,area_fraction,f_coriolis,i_min,i_max,j_min,j_max
0,0,0,0,-86.799751,-89.752815,-30.559948,-114.932281,0.000000,-0.000146,0,540,0,540
1,0,0,1,-87.051270,-89.997368,64.906548,-114.485664,0.000000,-0.000146,540,1080,0,540
2,0,0,2,-86.080391,-88.665024,64.958748,2.950904,0.000000,-0.000146,1080,1620,0,540
3,0,0,3,-84.818130,-87.089844,64.974991,25.209188,0.000000,-0.000145,1620,2160,0,540
4,0,0,4,-83.463402,-85.528854,64.983391,38.203091,0.000000,-0.000145,2160,2700,0,540
...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,12,7,3,-71.575722,-76.234230,-38.041668,-50.269459,1.000000,-0.000140,1620,2160,3780,4320
828,12,7,4,-76.018021,-80.353157,-38.619289,-53.336048,0.392284,-0.000143,2160,2700,3780,4320
829,12,7,5,-79.970764,-83.922943,-40.474148,-61.068802,0.000000,-0.000144,2700,3240,3780,4320
830,12,7,6,-83.296005,-86.793678,-45.510330,-79.909386,0.000000,-0.000145,3240,3780,3780,4320


In [5]:
regions_keep = region_df[region_df.area_fraction > 0.6]
regions_keep

,face,j,i,lat_max,lat_min,lon_max,lon_min,area_fraction,f_coriolis,i_min,i_max,j_min,j_max
32,0,4,0,-71.638092,-76.325882,-26.764791,-38.595600,0.995065,-0.000140,0,540,2160,2700
40,0,5,0,-67.254387,-71.654449,-26.760416,-38.020435,1.000000,-0.000136,0,540,2700,3240
41,0,5,1,-67.254387,-71.654922,-15.487511,-26.743858,1.000000,-0.000136,540,1080,2700,3240
42,0,5,2,-67.254387,-71.641052,-4.213777,-15.489583,0.928409,-0.000136,1080,1620,2700,3240
43,0,5,3,-67.254387,-71.589973,7.052837,-4.239583,0.760429,-0.000136,1620,2160,2700,3240
...,...,...,...,...,...,...,...,...,...,...,...,...,...
819,12,6,3,-71.481636,-76.009682,-49.313774,-61.779488,0.893937,-0.000140,1620,2160,3240,3780
824,12,7,0,-57.012096,-62.552940,-38.010418,-49.239582,0.994304,-0.000126,0,540,3780,4320
825,12,7,1,-62.562359,-67.246368,-38.010418,-49.239582,1.000000,-0.000132,540,1080,3780,4320
826,12,7,2,-67.254387,-71.629066,-38.010418,-49.292297,1.000000,-0.000136,1080,1620,3780,4320


In [6]:
def iter_regions():
    for idx in regions_keep.index:
        face, i_min, i_max, j_min, j_max = regions_keep[['face', 'i_min', 'i_max', 'j_min', 'j_max']].loc[idx]
        f = regions_keep.f_coriolis.loc[idx]
        j_slice = slice(j_min, j_max)
        i_slice = slice(i_min, i_max)
        region_selector = dict(face=2, j=j_slice, j_g=j_slice, i=i_slice, i_g=i_slice)
        yield idx, region_selector, f

In [7]:
for r, selector, f in iter_regions():
    break
r, selector, f

(32,
 {'face': 2,
  'j': slice(2160, 2700, None),
  'j_g': slice(2160, 2700, None),
  'i': slice(0, 540, None),
  'i_g': slice(0, 540, None)},
 -0.00014016188)

In [8]:
def iter_time(time_blocks=240, time_subset=24, ntot=9030):
    for nstart in range(0, ntot, time_blocks):
        if nstart + time_blocks <= ntot:
            yield slice(nstart, nstart+time_blocks, time_subset)

for time_slice in iter_time():
    pass

time_coarse = ds_grid.time.coarsen(time=240, boundary='trim').mean()
assert len(time_coarse) == len(list(iter_time()))

## Initialize Target Dataset



In [9]:
ntime = len(time_coarse)
nregions = len(regions_keep)
n_strain_bins = 100
n_vort_bins = 200

In [10]:
def vort_strain_bins(n_strain_bins, n_vort_bins, f=1, vort_max=7, strain_max=7):
    f_abs = abs(f)
    vort_bins = np.linspace(-vort_max * f_abs, vort_max * f_abs, n_vort_bins)
    strain_bins = np.linspace(0.0, strain_max * f_abs, n_strain_bins)
    return vort_bins, strain_bins
    
def center_bin(bounds):
    return 0.5 * (bounds[:-1] + bounds[1:])
    
vort_bins, strain_bins = vort_strain_bins(n_strain_bins, n_vort_bins)
vort_bins = center_bin(vort_bins)
strain_bins = center_bin(strain_bins)

In [11]:
import dask.array as dsa
ds_target = xr.DataArray(
    dsa.empty(
        shape=(ntime, nregions, n_strain_bins-1, n_vort_bins-1),
        chunks=(1, 1, n_strain_bins-1, n_vort_bins-1),
        dtype="f8"
    ),
    name="vort_strain_histogram",
    dims=['time_coarse', 'region', 'strain', 'vort'],
    coords={
        "time_coarse": time_coarse.values,
        "region": regions_keep.index.values,
        'vort': vort_bins,
        'strain': strain_bins
    }
).to_dataset()

ds_region = xr.Dataset(regions_keep.drop(columns=['i', 'j', 'face'])).rename({'dim_0': 'region'})
ds_target = ds_target.assign_coords(coords=ds_region)
ds_target

,Array,Chunk
Bytes,2.37 GiB,153.91 kiB
Shape,"(37, 437, 99, 199)","(1, 1, 99, 199)"
Count,16169 Tasks,16169 Chunks
Type,float64,numpy.ndarray


In [12]:
for c in ds_target.coords:
    ds_target[c].encoding['chunks'] = ds_target[c].shape

In [13]:
import os
target_url = os.environ['SCRATCH_BUCKET'] + '/llc4320/vort_strain_histogram.zarr'
target_url

'gcs://pangeo-integration-te-3eea-prod-scratch-bucket/yangleir/llc4320/vort_strain_histogram.zarr'

In [14]:
ds_target.to_zarr(target_url, compute=False, mode='w')

Delayed('_finalize_store-a8150dbc-355c-4096-a994-4fa90af2b005')

In [15]:
import zarr
zgroup = zarr.open_group(target_url)
zgroup.tree()

Tree(nodes=(Node(disabled=True, name='/', nodes=(Node(disabled=True, icon='table', name='area_fraction (437,) …

In [16]:
# We need xgcm >0.7.0 to get grid ufuncs
!pip install git+https://github.com/xgcm/xgcm.git

  Cloning https://github.com/xgcm/xgcm.git to /tmp/pip-req-build-t8zcg4no
  Running command git clone --filter=blob:none --quiet https://github.com/xgcm/xgcm.git /tmp/pip-req-build-t8zcg4no
  Resolved https://github.com/xgcm/xgcm.git to commit c66861d8ef3ad86acd1eaabff7c33033fbc53378
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for xgcm: filename=xgcm-0.8.1.dev34+gc66861d-py3-none-any.whl size=94925 sha256=7321a5518693915bdc6e89f87a95debeff0f07d9ef5991c15295e54906524fbc
  Stored in directory: /tmp/pip-ephem-wheel-cache-slw0s4nq/wheels/26/7a/09/292d90418050fbb90e9ea92a07bb96aec29bfe790cbdd2c396
Successfully built xgcm
  Attempting uninstall: xgcm
    Found existing installation: xgcm 0.6.1
    Uninstalling xgcm-0.6.1:
      Successfully uninstalled xgcm-0.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following depen

### Define calculation

We want to calculate the integral formula for [relative vorticity](https://mitgcm.readthedocs.io/en/latest/algorithm/algorithm.html?highlight=vorticity#relative-vorticity) in terms of circulation around a grid cell:

$$
  \zeta_3 = \frac{\Gamma}{A_\zeta} = \frac{1}{{\cal A}_\zeta} ( \delta_i \Delta y_c v - \delta_j \Delta x_c u )
$$

where $u$ and $v$ are horizontal and vertical components of surface velocity respectively.

In [17]:
import xgcm

def vort_ufunc(u, dxc, v, dyc, raz):
    u_trans = u * dxc
    v_trans = v * dyc
    v_diff_x = v_trans[..., 1:, 1:] - v_trans[..., 1:, :-1]
    u_diff_y = u_trans[..., 1:, 1:] - u_trans[..., :-1, 1:]
    return (v_diff_x - u_diff_y) / raz[..., 1:, 1:]


def vort(ds, grid):
    """Dimensionally-consistent relative vorticity"""

    u = ds.U
    v = ds.V
    dxC = grid._ds.dxC
    dyC = grid._ds.dyC
    rAz = grid._ds.rAz
       
    zeta = grid.apply_as_grid_ufunc(
        vort_ufunc,
        u, dxC, v, dyC, rAz,
        axis= 5 * [("Y", "X")],
        signature="(Y:center,X:left),(Y:center,X:left),(Y:left,X:center),(Y:left,X:center),(Y:left,X:left)->(Y:left,X:left)",
        boundary_width={"X": (1, 0), "Y": (1, 0)},
        boundary="fill",
        fill_value=np.nan,
        dask="forbidden",   # data should have been loaded in already
    )
    
    return zeta

We also want the **strain magnitude**, given by the vector magnitude of the two strain components, normal strain ($\sigma_n$) and shear strain ($\sigma_s$)

$$
\sigma = \sqrt{\sigma_n^2 + \sigma_s^2}
$$

where the components are given in terms of surface velocity components $u$ and $v$ as

$$
\sigma_n = \frac{\partial u}{\partial x} - \frac{\partial v}{\partial y}
$$
$$
\sigma_s = \frac{\partial v}{\partial x} - \frac{\partial u}{\partial y}
$$

In [18]:
def strain_ufunc(u, dxc, v, dyc):
    
    # TODO are these derivatives actually correct??
    u_diff_x = (u[..., 1:] - u[..., :-1]) / dxc[..., 1:]
    v_diff_x = (v[..., 1:] - v[..., :-1]) / dxc[..., 1:]
    u_diff_y = (u[..., 1:, :] - u[..., :-1, :]) / dyc[..., 1:, :]
    v_diff_y = (v[..., 1:, :] - v[..., :-1, :]) / dyc[..., 1:, :]
    
    u_diff_x_trimmed = u_diff_x[..., 1:, :]
    v_diff_x_trimmed = v_diff_x[..., 1:, :]
    u_diff_y_trimmed = u_diff_y[..., 1:]
    v_diff_y_trimmed = v_diff_y[..., 1:]
    
    strain_normal = u_diff_x_trimmed - v_diff_y_trimmed
    strain_shear = v_diff_x_trimmed + u_diff_y_trimmed
    
    strain_mag = np.sqrt(strain_normal ** 2 +  strain_shear ** 2)
    
    return strain_mag
    

def strain(ds, grid):
    """Strain magnitude"""
    
    u = ds.U
    v = ds.V
    dxC = grid._ds.dxC
    dyC = grid._ds.dyC
    
    sigma = grid.apply_as_grid_ufunc(
        strain_ufunc,
        u, dxC, v, dyC,
        axis= 4 * [("Y", "X")],
        signature="(Y:center,X:left),(Y:center,X:left),(Y:left,X:center),(Y:left,X:center)->(Y:left,X:left)",
        boundary_width={"X": (1, 0), "Y": (1, 0)},
        boundary="fill",
        fill_value=np.nan,
        dask="forbidden",   # data should have been loaded in already
    )
    
    return sigma

In [19]:
from xhistogram.xarray import histogram

def hist(omega, sigma, f, n_strain_bins, n_vort_bins):

    vort_bins, strain_bins = vort_strain_bins(n_strain_bins, n_vort_bins)
        
    vort = omega.rename('vort') / f
    strain = sigma.rename('strain') / abs(f)
    return histogram(
        vort,
        strain, 
        dim=['i_g', 'j_g'], 
        bins=[vort_bins, strain_bins],
        block_size=None,  # avoids a divide-by-zero bug in xhistogram's code for automatically determining block_size
        density=True,  # we don't care about number of grid points, so plot probability density
        keep_coords=True,
    )


## Compute Data

In [20]:
def open_llc4320_velocity_data(chunks=None):
    
    ds_SSU = xr.open_dataset(
        "gs://pangeo-ecco-llc4320/ssu", 
        engine="zarr", 
        storage_options={'requester_pays': True}, 
        chunks=chunks, 
        consolidated=True, 
    )
    
    ds_SSV = xr.open_dataset(
        "gs://pangeo-ecco-llc4320/ssv", 
        engine="zarr", 
        storage_options={'requester_pays': True}, 
        chunks=chunks, 
        consolidated=True, 
    )
    
    # Open grid data
    ds_grid = xr.open_dataset(
        "gs://pangeo-ecco-llc4320/grid", 
        engine="zarr", 
        storage_options={'requester_pays': True}, 
        chunks=chunks, 
        consolidated=True, 
    )
    
    coords_to_keep = ['dxC', 'dyC', 'rAz']
    ds_grid = ds_grid.reset_coords()[coords_to_keep]
    
    ds = xr.merge([ds_SSU, ds_SSV, ds_grid])
    return ds

In [21]:
def store_hist(h, ntime, nregion, target_url):
    h_mean = h.mean('time').rename("vort_strain_histogram").to_dataset()
    h_mean = h_mean.expand_dims(['region', 'time_coarse'])
    h_mean = h_mean.rename({"strain_bin": "strain", "vort_bin": "vort"}).drop("face")
    h_mean = h_mean.transpose('time_coarse', 'region', 'strain', 'vort')
    h_mean.to_zarr(
        target_url,
        region={
            "time_coarse": slice(ntime, ntime+1),
            "region": slice(nregion, nregion+1),
            "vort": slice(None, None),
            "strain": slice(None, None)
        }
    )

In [22]:
def calc_and_store_histogram(ntime, time_slice, nregion, region_slice, f, n_strain_bins, n_vort_bins, target_url):
    ds = open_llc4320_velocity_data()
    ds_reg = ds.isel(time=time_slice).isel(**region_slice)
    ds_reg.load()
    grid = xgcm.Grid(ds_reg, periodic=False)
    omega = vort(ds_reg, grid)
    sigma = strain(ds_reg, grid)    
    h = hist(omega, sigma, f, n_strain_bins, n_vort_bins)
    store_hist(h, ntime, nregion, target_url)

In [23]:
import dask
@dask.delayed
def calc_and_store_histogram_delayed(ntime, time_slice, nregion, region_slice, f, n_strain_bins, n_vort_bins, target_url):
    calc_and_store_histogram(ntime, time_slice, nregion, region_slice, f, n_strain_bins, n_vort_bins, target_url)

In [24]:
all_calcs = []
for ntime, time_slice in enumerate(iter_time()):
    for nregion, (region, region_slice, f) in enumerate(iter_regions()):
        all_calcs.append(
            calc_and_store_histogram_delayed(ntime, time_slice, nregion, region_slice, f, n_strain_bins, n_vort_bins, target_url)
        )
len(all_calcs)

16169